In [71]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import lit, rand, when, col, array, element_at
import random

In [72]:
# Inicializar SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Exercicio Intro") \
    .getOrCreate()

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
# Etapa 2
#  Renomeando a primeira coluna do df para "nomes"
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")

# Verificar o schema após a renomeação
df_nomes.printSchema()

# Mostrar as primeiras 10 linhas do DataFrame renomeado
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [ ]:
# Etapa 3
# Lista de escolaridades
escolaridades = ["Fundamental", "Medio", "Superior"]

# Adicionar a coluna Escolaridade com valores aleatórios
df_nomes = df_nomes.withColumn("Escolaridade",
                               when(rand() < 0.33, "Fundamental")
                               .when(rand() < 0.66, "Medio")
                               .otherwise("Superior"))

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet| Fundamental|
|    Jamie Russell|       Medio|
|   Edward Kistler|    Superior|
|    Sheila Maurer|       Medio|
| Donald Golightly|       Medio|
|       David Gray| Fundamental|
|      Joy Bennett| Fundamental|
|      Paul Kriese| Fundamental|
|Berniece Ornellas| Fundamental|
|    Brian Farrell| Fundamental|
+-----------------+------------+
only showing top 10 rows



In [ ]:
# Etapa 4
# Definir a lista de países
paises = ["Brasil", "Argentina", "Chile", "Peru", "Colombia"]

# Adicionar coluna de País com valores aleatórios
df_nomes = df_nomes.withColumn("Pais",
                               when(rand() < 0.2, "Brasil")
                               .when(rand() < 0.4, "Argentina")
                               .when(rand() < 0.6, "Chile")
                               .when(rand() < 0.8, "Peru")
                               .otherwise("Colombia"))

# Mostrar as primeiras 10 linhas para verificar a distribuição
df_nomes.show(10)

+-----------------+------------+---------+
|            Nomes|Escolaridade|     Pais|
+-----------------+------------+---------+
|   Frances Bennet| Fundamental|    Chile|
|    Jamie Russell|       Medio|   Brasil|
|   Edward Kistler|    Superior|Argentina|
|    Sheila Maurer|       Medio| Colombia|
| Donald Golightly|       Medio|     Peru|
|       David Gray| Fundamental|    Chile|
|      Joy Bennett| Fundamental|    Chile|
|      Paul Kriese| Fundamental|    Chile|
|Berniece Ornellas| Fundamental|     Peru|
|    Brian Farrell| Fundamental|     Peru|
+-----------------+------------+---------+
only showing top 10 rows



In [ ]:
# Etapa 5
# Adicionar coluna de Ano de Nascimento com valores aleatórios entre 1945 e 2010
df_nomes = df_nomes.withColumn("AnoNascimento", (rand() * (2010 - 1945) + 1945).cast("int"))

# Mostrar as primeiras 10 linhas do DataFrame atualizado
df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet| Fundamental|    Chile|         1996|
|    Jamie Russell|       Medio|   Brasil|         1963|
|   Edward Kistler|    Superior|Argentina|         1997|
|    Sheila Maurer|       Medio| Colombia|         1982|
| Donald Golightly|       Medio|     Peru|         2000|
|       David Gray| Fundamental|    Chile|         1998|
|      Joy Bennett| Fundamental|    Chile|         2005|
|      Paul Kriese| Fundamental|    Chile|         1991|
|Berniece Ornellas| Fundamental|     Peru|         1973|
|    Brian Farrell| Fundamental|     Peru|         2009|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [ ]:
# Etapa 6
# Selecionar pessoas nascidas a partir do ano 2000 usando o select do dataframe
df_select = df_nomes.filter(df_nomes.AnoNascimento >= 2000)

# Mostrar as primeiras 10 linhas
df_select.show(10)


+--------------------+------------+---------+-------------+
|               Nomes|Escolaridade|     Pais|AnoNascimento|
+--------------------+------------+---------+-------------+
|    Donald Golightly|       Medio|     Peru|         2000|
|         Joy Bennett| Fundamental|    Chile|         2005|
|       Brian Farrell| Fundamental|     Peru|         2009|
|         Albert Leef| Fundamental|Argentina|         2007|
|        Charles Hill|    Superior|Argentina|         2007|
|Christopher Williams| Fundamental|    Chile|         2001|
|           Ana Baker| Fundamental|     Peru|         2001|
|   Marguerite Rankin| Fundamental|    Chile|         2001|
|       Jenna Johnson|       Medio|   Brasil|         2005|
|     Suzanne Bullard|    Superior|     Peru|         2003|
+--------------------+------------+---------+-------------+
only showing top 10 rows



In [ ]:
# Etapa 7
# Registrar a tabela temporária
df_nomes.createOrReplaceTempView("pessoas")

# Executar consulta SQL com o select SQL
df_select_sql = spark.sql("SELECT * FROM pessoas WHERE AnoNascimento >= 2000")

# Mostrar as primeiras 10 linhas
df_select_sql.show(10)


+--------------------+------------+---------+-------------+
|               Nomes|Escolaridade|     Pais|AnoNascimento|
+--------------------+------------+---------+-------------+
|    Donald Golightly|       Medio|     Peru|         2000|
|         Joy Bennett| Fundamental|    Chile|         2005|
|       Brian Farrell| Fundamental|     Peru|         2009|
|         Albert Leef| Fundamental|Argentina|         2007|
|        Charles Hill|    Superior|Argentina|         2007|
|Christopher Williams| Fundamental|    Chile|         2001|
|           Ana Baker| Fundamental|     Peru|         2001|
|   Marguerite Rankin| Fundamental|    Chile|         2001|
|       Jenna Johnson|       Medio|   Brasil|         2005|
|     Suzanne Bullard|    Superior|     Peru|         2003|
+--------------------+------------+---------+-------------+
only showing top 10 rows



In [ ]:
# Etapa 8
# Filtrar e contar as pessoas da geração Millennials (nascidos entre 1980 e 1994) no dataset
millennials_count = df_nomes.select("AnoNascimento").filter((col("AnoNascimento") >= 1980) & (col("AnoNascimento") <= 1994)).count()

# Exibir o resultado
print(f"Total de pessoas da geração Millennials: {millennials_count}")

Total de pessoas da geração Millennials: 2306756


In [ ]:
# Etapa 9
# Registrar a tabela temporária
df_nomes.createOrReplaceTempView("contagem")

# Executar a consulta SQL para contar as pessoas da geração Millennials
millennials_count_sql = spark.sql("""
    SELECT COUNT(*) as Total
    FROM contagem
    WHERE AnoNascimento BETWEEN 1980 AND 1994
""").collect()[0]['Total']

# Exibir o resultado
print(f"Total de pessoas da geração Millennials: {millennials_count_sql}")

Total de pessoas da geração Millennials: 2306756


In [ ]:
# Adicionar coluna de Geração usando a função `when`
df_nomes = df_nomes.withColumn("Geração", 
                               when(col("AnoNascimento").between(1944, 1964), "Baby Boomers")
                               .when(col("AnoNascimento").between(1965, 1979), "Geração X")
                               .when(col("AnoNascimento").between(1980, 1994), "Millennials")
                               .when(col("AnoNascimento").between(1995, 2015), "Geração Z"))


In [ ]:
# Registrar a tabela temporária com o nome "geracao"
df_nomes.createOrReplaceTempView("geracao")

# Executar a consulta SQL para contar as pessoas da geração Millennials
millennials_count_sql = spark.sql("""
    SELECT COUNT(*) as Total
    FROM geracao
    WHERE AnoNascimento BETWEEN 1980 AND 1994
""").collect()[0]['Total']

# Mostrar o resultado
resultado.show()


ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'ç'.(line 4, pos 12)

== SQL ==

    SELECT 
        Pais, 
        Geração, 
------------^^^
        COUNT(*) as Quantidade
    FROM (
        SELECT 
            Pais,
            CASE
                WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
                WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geração X'
                WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
                WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Geração Z'
            END AS Geração
        FROM geracao
    ) subquery
    GROUP BY Pais, Geração
    ORDER BY Pais ASC, Geração ASC, Quantidade ASC


24/08/17 20:49:54 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 931263 ms exceeds timeout 120000 ms
24/08/17 20:49:54 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/17 20:49:58 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$